In [1]:
import pandas as pd
import os  
from datetime import timedelta, datetime
from pathlib import Path

from fastf1.ergast import Ergast
import fastf1.plotting

ergast = Ergast()

#parent_file = Path(__file__).resolve().parent.parent
parent_file = '/home/kurios/Documents/f1_analysis/'

year = 2025
race_from = 1
race_to = 7

report_folder = parent_file + 'reports'

In [2]:
def parse_laptime_str(lap_string):
    """Parses a lap time string (MM:SS.fff) into a timedelta object."""
    if isinstance(lap_string, str) and ':' in lap_string and '.' in lap_string:
        minutes, rest = lap_string.split(':')
        seconds, milliseconds = rest.split('.')
        return timedelta(minutes=int(minutes), seconds=int(seconds), milliseconds=int(milliseconds))
    else:
        return timedelta(0)
    

def Hex_RGB(ip):
    return tuple(int(ip[i+1:i+3],16) for i in (0, 2, 4))

In [ ]:
   
race_years = list(range(race_from, race_to+1))
recaps = {}
zero_time = timedelta(0)
threshold_multiplier = 1.5

qual_sessions = range(1, 4)


for race_number in race_years:
    for race_session in ('S', 'R', 'SQ', 'Q'):
        try:
            session = fastf1.get_session(year, race_number, race_session)
            session.load()
            print(session)
            teams = fastf1.plotting.list_team_names(session)
        except:
            continue
        if race_session == 'S' or race_session == 'R':
            keyword = f'{race_number}_{race_session}_race_info'
            driver_data_file = None
            for fname in os.listdir(parent_file + 'data/processed/'):
                if keyword in fname:
                    driver_data_file = fname
                    break

            if driver_data_file:
                os.chdir(parent_file + 'data/processed/')
                arr = pd.read_csv(driver_data_file)
                num_rows = len(arr)

                temp_recap = {}

                for i in range(num_rows):
                    name_driver_1 = arr['driver_1_name'][i]
                    name_driver_2 = arr['driver_2_name'][i]

                    name = f'{name_driver_1}_{name_driver_2}'

                    iqr_driver_1 = float(arr['iqr_driver_1'][i][:-2])
                    iqr_driver_2 = float(arr['iqr_driver_2'][i][:-2])

                    laptime_driver_1_str = arr['avg_laptime_driver_1'][i]
                    laptime_driver_2_str = arr['avg_laptime_driver_2'][i]

                    fastest_lap_driver_1_str = arr['fastest_driver_1'][i]
                    fastest_lap_driver_2_str = arr['fastest_driver_2'][i]
                    safety_car_lap_str = arr['safety_car_lap'][i]

                    avg_seconds_driver_1 = parse_laptime_str(laptime_driver_1_str)
                    avg_seconds_driver_2 = parse_laptime_str(laptime_driver_2_str)

                    if (iqr_driver_1 != 0 and iqr_driver_2 != 0 and avg_seconds_driver_1 != zero_time and avg_seconds_driver_2 != zero_time
                            and avg_seconds_driver_1 <= threshold_multiplier * avg_seconds_driver_2 and avg_seconds_driver_2 <= threshold_multiplier * avg_seconds_driver_1):

                        if datetime.strptime(laptime_driver_1_str, '%M:%S.%f') < datetime.strptime(laptime_driver_2_str, '%M:%S.%f'):
                            gap_driver_1 = -round((pd.to_timedelta(datetime.strptime(laptime_driver_2_str, '%M:%S.%f') - datetime.strptime(laptime_driver_1_str, '%M:%S.%f')).total_seconds()), 3)
                        else:
                            gap_driver_1 = round((pd.to_timedelta(datetime.strptime(laptime_driver_1_str, '%M:%S.%f') - datetime.strptime(laptime_driver_2_str, '%M:%S.%f')).total_seconds()), 3)

                        gap_driver_2 = -gap_driver_1

                        lap_advantage_1 = round((int(fastest_lap_driver_1_str[:-3])/(int(fastest_lap_driver_1_str[-2:])-int(safety_car_lap_str[:-3])))*100)
                        lap_advantage_2 = round((int(fastest_lap_driver_2_str[:-3])/(int(fastest_lap_driver_2_str[-2:])-int(safety_car_lap_str[:-3])))*100)

                        pit_stop_number_driver_1 = int(arr['number_pit_driver_1'][i])
                        pit_stop_number_driver_2 = int(arr['number_pit_driver_2'][i])

                        pit_stop_time_driver_1_td = parse_laptime_str(arr['total_duration_pit_driver_1'][i][:-2])
                        pit_stop_time_driver_2_td = parse_laptime_str(arr['total_duration_pit_driver_2'][i][:-2])

                        temp_recap.setdefault(name, {'first_race': 0, 'last_race': 0, 'total_iqr_driver_1': 0, 'total_gap_driver_1': 0, 'total_lap_advantage_driver_1': 0, 'total_pit_stop_time_driver_1': timedelta(0), 'total_pit_stop_number_driver_1': 0, 'count_driver_1': 0, 'total_pit_stop_number Driver 1': 0, 'total_qual_gap Driver 1': 0, 'total_corner_advantage Driver 1': 0, 'qual_count Driver 1': 0, 
                                                    'total_iqr_driver_2': 0, 'total_gap_driver_2': 0, 'total_lap_advantage_driver_2': 0, 'total_pit_stop_time_driver_2': timedelta(0), 'total_pit_stop_number_driver_2': 0, 'count_driver_2': 0, 'total_pit_stop_number Driver 2': 0, 'total_qual_gap Driver 2': 0, 'total_corner_advantage Driver 2': 0, 'qual_count Driver 2': 0})
                        
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['first_race'] = min(race_number, temp_recap[f'{name_driver_1}_{name_driver_2}']['first_race'])
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['last_race'] = max(race_number, temp_recap[f'{name_driver_1}_{name_driver_2}']['last_race'])

                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_iqr_driver_1'] += iqr_driver_1
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_gap_driver_1'] += gap_driver_1
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_lap_advantage_driver_1'] += lap_advantage_1
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_pit_stop_time_driver_1'] += pit_stop_time_driver_1_td
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_pit_stop_number_driver_1'] += pit_stop_number_driver_1
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['count_driver_1'] += 1
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_qual_gap Driver 1'] = 0
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_corner_advantage Driver 1'] = 0

                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_iqr_driver_2'] += iqr_driver_2
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_gap_driver_2'] += gap_driver_2
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_lap_advantage_driver_2'] += lap_advantage_2
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_pit_stop_time_driver_2'] += pit_stop_time_driver_2_td
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_pit_stop_number_driver_2'] += pit_stop_number_driver_2
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['count_driver_2'] += 1
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_qual_gap Driver 2'] = 0
                        temp_recap[f'{name_driver_1}_{name_driver_2}']['total_corner_advantage Driver 2'] = 0 

                for name, data in temp_recap.items():

                    recaps.setdefault(name, {'First Race': 0, 'Last Race': 0, 'Average IQR Driver 1': 0, 'Average Gap Race Lap Driver 1': 0, 'Lap Advantage Driver 1': 0, 'Average Pit Stop Time Driver 1': timedelta(0), 'total_count Driver 1': 0, 'total_pit_stop_number Driver 1': 0, 'total_qual_gap Driver 1': 0, 'total_corner_advantage Driver 1': 0, 'qual_count Driver 1': 0, 
                                            'Average IQR Driver 2': 0, 'Average Gap Race Lap Driver 2': 0, 'Lap Advantage Driver 2': 0, 'Average Pit Stop Time Driver 2': timedelta(0), 'total_count Driver 2': 0, 'total_pit_stop_number Driver 2': 0, 'total_qual_gap Driver 2': 0, 'total_corner_advantage Driver 2': 0, 'qual_count Driver 2': 0})
                    
                    recaps[name]['First Race'] = data['first_race'] + 1
                    recaps[name]['Last Race'] = data['last_race']

                    recaps[name]['Average IQR Driver 1'] += data['total_iqr_driver_1']
                    recaps[name]['Average Gap Race Lap Driver 1'] += data['total_gap_driver_1']
                    recaps[name]['Lap Advantage Driver 1'] += data['total_lap_advantage_driver_1']
                    recaps[name]['Average Pit Stop Time Driver 1'] += data['total_pit_stop_time_driver_1']
                    recaps[name]['total_count Driver 1'] += data['count_driver_1']
                    recaps[name]['total_pit_stop_number Driver 1'] += data['total_pit_stop_number_driver_1']
                    
                    recaps[name]['Average IQR Driver 2'] += data['total_iqr_driver_2']
                    recaps[name]['Average Gap Race Lap Driver 2'] += data['total_gap_driver_2']
                    recaps[name]['Lap Advantage Driver 2'] += data['total_lap_advantage_driver_2']
                    recaps[name]['Average Pit Stop Time Driver 2'] += data['total_pit_stop_time_driver_2']
                    recaps[name]['total_count Driver 2'] += data['count_driver_2']
                    recaps[name]['total_pit_stop_number Driver 2'] += data['total_pit_stop_number_driver_2']
        
        if race_session == 'SQ' or race_session == 'Q':            
            for qual_session in qual_sessions:
                keyword = f'{race_number}_{race_session}_drivers_info_Q{qual_session}'
                driver_data_file = None
                for fname in os.listdir(parent_file + 'data/processed/'):
                    if keyword in fname:
                        driver_data_file = fname
                        break

                if driver_data_file:
                    os.chdir(parent_file + 'data/processed/')
                    arr = pd.read_csv(driver_data_file)

                    num_rows = len(arr)

                    for i in range(num_rows):
                        name_driver_1 = arr['Name'][i]
                        name_driver_2 = arr['Name_1'][i]

                        name = f'{name_driver_1}_{name_driver_2}'

                        laptime_driver_1_str = arr['LapTime'][i]
                        laptime_driver_2_str = arr['LapTime_1'][i]

                        avg_seconds_driver_1 = parse_laptime_str(laptime_driver_1_str)
                        avg_seconds_driver_2 = parse_laptime_str(laptime_driver_2_str)

                        if (avg_seconds_driver_1 != zero_time and avg_seconds_driver_2 != zero_time
                                and avg_seconds_driver_1 <= threshold_multiplier * avg_seconds_driver_2 and avg_seconds_driver_2 <= threshold_multiplier * avg_seconds_driver_1):

                            time_driver_1 = datetime.strptime(laptime_driver_1_str, '%M:%S.%f')
                            time_driver_2 = datetime.strptime(laptime_driver_2_str, '%M:%S.%f')

                            gap = round((pd.to_timedelta(time_driver_1 - time_driver_2).total_seconds()), 3)

                            corner_adv_1_num, corner_adv_1_den = map(int, arr['corner_advantage'][i].split('/'))
                            corner_advantage_driver_1 = (corner_adv_1_num / corner_adv_1_den) * 100

                            corner_adv_2_num, corner_adv_2_den = map(int, arr['corner_advantage1'][i].split('/'))
                            corner_advantage_driver_2 = (corner_adv_2_num / corner_adv_2_den) * 100
                            
                            if not name in temp_recap:
                                temp_recap.setdefault(name, {'Average IQR Driver 1': 0, 'Average Gap Race Lap Driver 1': 0, 'Lap Advantage Driver 1': 0, 'Average Pit Stop Time Driver 1': timedelta(0), 'total_count Driver 1': 0, 'total_pit_stop_number Driver 1': 0, 'total_qual_gap Driver 1': 0, 'total_corner_advantage Driver 1': 0, 'qual_count Driver 1': 0, 
                                                            'Average IQR Driver 2': 0, 'Average Gap Race Lap Driver 2': 0, 'Lap Advantage Driver 2': 0, 'Average Pit Stop Time Driver 2': timedelta(0), 'total_count Driver 2': 0, 'total_pit_stop_number Driver 2': 0, 'total_qual_gap Driver 2': 0, 'total_corner_advantage Driver 2': 0, 'qual_count Driver 2': 0})
                            
                            temp_recap[f'{name_driver_1}_{name_driver_2}']['total_qual_gap Driver 1'] += gap
                            temp_recap[f'{name_driver_1}_{name_driver_2}']['total_corner_advantage Driver 1'] += corner_advantage_driver_1
                            temp_recap[f'{name_driver_1}_{name_driver_2}']['qual_count Driver 1'] += 1

                            temp_recap[f'{name_driver_1}_{name_driver_2}']['total_qual_gap Driver 2'] -= gap
                            temp_recap[f'{name_driver_1}_{name_driver_2}']['total_corner_advantage Driver 2'] += corner_advantage_driver_2
                            temp_recap[f'{name_driver_1}_{name_driver_2}']['qual_count Driver 2'] += 1

                    for name, data in temp_recap.items():
                        
                        if not name in recaps:
                            recaps.setdefault(name, {'Average IQR Driver 1': 0, 'Average Gap Race Lap Driver 1': 0, 'Lap Advantage Driver 1': 0, 'Average Pit Stop Time Driver 1': timedelta(0), 'total_count Driver 1': 0, 'total_pit_stop_number Driver 1': 0, 'total_qual_gap Driver 1': 0, 'total_corner_advantage Driver 1': 0, 'qual_count Driver 1': 0, 
                                                     'Average IQR Driver 2': 0, 'Average Gap Race Lap Driver 2': 0, 'Lap Advantage Driver 2': 0, 'Average Pit Stop Time Driver 2': timedelta(0), 'total_count Driver 2': 0, 'total_pit_stop_number Driver 2': 0, 'total_qual_gap Driver 2': 0, 'total_corner_advantage Driver 2': 0, 'qual_count Driver 2': 0})
                        
                        recaps[name]['total_qual_gap Driver 1'] += data['total_qual_gap Driver 1']
                        recaps[name]['total_corner_advantage Driver 1'] += data['total_corner_advantage Driver 1']
                        recaps[name]['qual_count Driver 1'] += data['qual_count Driver 1']

                        recaps[name]['total_qual_gap Driver 2'] += data['total_qual_gap Driver 2']
                        recaps[name]['total_corner_advantage Driver 2'] += data['total_corner_advantage Driver 2']
                        recaps[name]['qual_count Driver 2'] += data['qual_count Driver 2']


final_race_recaps = {}
for name, data in recaps.items():
    
    first_race = data['First Race']
    last_race = data['Last Race']

    count_driver_1 = data['total_count Driver 1']
    avg_iqr_driver_1 = round(data['Average IQR Driver 1'] / count_driver_1, 3) if count_driver_1 > 0 else "N/A"
    avg_gap_driver_1 = round(data['Average Gap Race Lap Driver 1'] / count_driver_1, 3) if count_driver_1 > 0 else "N/A"
    lap_advantage_driver_1 = round(data['Lap Advantage Driver 1'] / count_driver_1) if count_driver_1 > 0 else "N/A"
    avg_pit_stop_time_driver_1 = round(data['Average Pit Stop Time Driver 1'].total_seconds() / data['total_pit_stop_number Driver 1'], 3) if data['total_pit_stop_number Driver 1'] > 0 else "N/A"

    count_qual_driver_1 = data['qual_count Driver 1']
    avg_qual_gap_driver_1 = round(data['total_qual_gap Driver 1'] / count_qual_driver_1, 3) if count_qual_driver_1 > 0 else "N/A"
    avg_corner_advantage_driver_1 = round(data['total_corner_advantage Driver 1'] / count_qual_driver_1) if count_qual_driver_1 > 0 else "N/A"
    
    count_driver_2 = data['total_count Driver 2']
    avg_iqr_driver_2 = round(data['Average IQR Driver 2'] / count_driver_2, 3) if count_driver_2 > 0 else "N/A"
    avg_gap_driver_2 = round(data['Average Gap Race Lap Driver 2'] / count_driver_2, 3) if count_driver_2 > 0 else "N/A"
    lap_advantage_driver_2 = round(data['Lap Advantage Driver 2'] / count_driver_2) if count_driver_2 > 0 else "N/A"
    avg_pit_stop_time_driver_2 = round(data['Average Pit Stop Time Driver 2'].total_seconds() / data['total_pit_stop_number Driver 2'], 3) if data['total_pit_stop_number Driver 2'] > 0 else "N/A"

    count_qual_driver_2 = data['qual_count Driver 2']
    avg_qual_gap_driver_2 = round(data['total_qual_gap Driver 2'] / count_qual_driver_2, 3) if count_qual_driver_2 > 0 else "N/A"
    avg_corner_advantage_driver_2 = round(data['total_corner_advantage Driver 2'] / count_qual_driver_2) if count_qual_driver_2 > 0 else "N/A"

    final_race_recaps[name] = {
        'First Race' : first_race,
        'Last Race' : last_race,
        'Average Gap Quali Lap Driver 1': f'{avg_qual_gap_driver_1}s',
        'Corner Advantage Driver 1': f'{avg_corner_advantage_driver_1}%',
        'Average IQR Driver 1': f'{avg_iqr_driver_1}s',
        'Average Gap Race Lap Driver 1': f'{avg_gap_driver_1}s',
        'Lap Advantage Driver 1': f'{lap_advantage_driver_1}%',
        'Average Pit Stop Time Driver 1': f'{avg_pit_stop_time_driver_1}s',

        'Average Gap Quali Lap Driver 2': f'{avg_qual_gap_driver_2}s',
        'Corner Advantage Driver 2': f'{avg_corner_advantage_driver_2}%',
        'Average IQR Driver 2': f'{avg_iqr_driver_2}s',
        'Average Gap Race Lap Driver 2': f'{avg_gap_driver_2}s',
        'Lap Advantage Driver 2': f'{lap_advantage_driver_2}%',
        'Average Pit Stop Time Driver 2': f'{avg_pit_stop_time_driver_2}s'
    }

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', 

2025 Season Round 1: Australian Grand Prix - Race


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '1', '63', '22', '23', '16', '44', '10', '55', '6', '14', '18', '7', '5', '12', '27', '30', '31', '87']
core           INFO 	Loading data for Chinese Grand Prix - Sprint [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


2025 Season Round 1: Australian Grand Prix - Qualifying


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 44 completed the race distance 00:00.020000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['44', '81', '1', '63', '16', '22', '12', '4', '18', '14', '23', '10', '6', '30', '87', '31', '55', '5', '27', '7']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_dat

...

2025 Season Round 6: Miami Grand Prix - Qualifying


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '44', '23', '16', '63', '55', '6', '22', '14', '27', '10', '30', '18', '43', '87', '5', '12', '31']
core           INFO 	Loading data for Emilia Romagna Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


2025 Season Round 7: Emilia Romagna Grand Prix - Race


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '1', '63', '4', '14', '55', '23', '18', '6', '10', '16', '44', '12', '5', '43', '30', '27', '31', '87', '22']


2025 Season Round 7: Emilia Romagna Grand Prix - Qualifying


In [24]:
print(final_race_recaps)

{'Max Verstappen_Liam Lawson': {'First Race': 1, 'Last Race': 2, 'Average Gap Quali Lap Driver 1': '-0.863s', 'Corner Advantage Driver 1': '74%', 'Average IQR Driver 1': '2.308s', 'Average Gap Race Lap Driver 1': '-1.208s', 'Lap Advantage Driver 1': '86%', 'Average Pit Stop Time Driver 1': '16.662s', 'Average Gap Quali Lap Driver 2': '0.863s', 'Corner Advantage Driver 2': '26%', 'Average IQR Driver 2': '2.267s', 'Average Gap Race Lap Driver 2': '1.208s', 'Lap Advantage Driver 2': '14%', 'Average Pit Stop Time Driver 2': '19.156s'}, 'Kimi Antonelli_George Russell': {'First Race': 1, 'Last Race': 7, 'Average Gap Quali Lap Driver 1': '0.12s', 'Corner Advantage Driver 1': '50%', 'Average IQR Driver 1': '1.724s', 'Average Gap Race Lap Driver 1': '0.472s', 'Lap Advantage Driver 1': '26%', 'Average Pit Stop Time Driver 1': '24.487s', 'Average Gap Quali Lap Driver 2': '-0.12s', 'Corner Advantage Driver 2': '50%', 'Average IQR Driver 2': '1.554s', 'Average Gap Race Lap Driver 2': '-0.472s', 'La

In [15]:
print(recaps)

{'Max Verstappen_Liam Lawson': {'Average IQR Driver 1': 6.923, 'Average Gap Race Lap Driver 1': -3.625, 'Lap Advantage Driver 1': 257, 'Average Pit Stop Time Driver 1': datetime.timedelta(seconds=99, microseconds=969000), 'total_count Driver 1': 3, 'total_pit_stop_number Driver 1': 6, 'total_qual_gap Driver 1': -10.356, 'total_corner_advantage Driver 1': 892.9411764705885, 'qual_count Driver 1': 12, 'Average IQR Driver 2': 6.800000000000001, 'Average Gap Race Lap Driver 2': 3.625, 'Lap Advantage Driver 2': 43, 'Average Pit Stop Time Driver 2': datetime.timedelta(seconds=114, microseconds=937000), 'total_count Driver 2': 3, 'total_pit_stop_number Driver 2': 6, 'total_qual_gap Driver 2': 10.356, 'total_corner_advantage Driver 2': 307.05882352941177, 'qual_count Driver 2': 12}, 'Kimi Antonelli_George Russell': {'Average IQR Driver 1': 15.517000000000003, 'Average Gap Race Lap Driver 1': 4.251, 'Lap Advantage Driver 1': 238, 'Average Pit Stop Time Driver 1': datetime.timedelta(seconds=195,

In [8]:
print(temp_recap)

{'Max Verstappen_Yuki Tsunoda': {'total_iqr_driver_1': 1.606, 'total_gap_driver_1': -1.261, 'total_lap_advantage_driver_1': 96, 'total_pit_stop_time_driver_1': datetime.timedelta(seconds=59, microseconds=648000), 'total_pit_stop_number_driver_1': 2, 'count_driver_1': 1, 'total_pit_stop_number Driver 1': 0, 'total_qual_gap Driver 1': 0, 'total_corner_advantage Driver 1': 0, 'qual_count Driver 1': 0, 'total_iqr_driver_2': 1.463, 'total_gap_driver_2': 1.261, 'total_lap_advantage_driver_2': 4, 'total_pit_stop_time_driver_2': datetime.timedelta(seconds=29, microseconds=807000), 'total_pit_stop_number_driver_2': 1, 'count_driver_2': 1, 'total_pit_stop_number Driver 2': 0, 'total_qual_gap Driver 2': 0, 'total_corner_advantage Driver 2': 0, 'qual_count Driver 2': 0}, 'Pierre Gasly_Franco Colapinto': {'total_iqr_driver_1': 1.259, 'total_gap_driver_1': -0.604, 'total_lap_advantage_driver_1': 72, 'total_pit_stop_time_driver_1': datetime.timedelta(seconds=63, microseconds=491000), 'total_pit_stop_